In [1]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import matplotlib.pyplot as plt
from imp import reload
from typing import List
import os
import pickle
import pandas as pd
import numpy as np
import sys
from skimage.transform import resize

from data_utils import DataHandler
import evaluate
import text_retrieval
import image_features
import image_utils
import bg_masks
import similarities
import detect_textbox
import histogram_processing
import visualizer

N_PROCESS = 10

/tmp/ipykernel_25012/3934259341.py:4: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


# LOAD DATA

In [2]:
data_handler = DataHandler()

# Load images:
db, db_files = data_handler.load_images(folder = "../data/BBDD/", extension = ".jpg", desc = "Loading BBDD Data...")
qst1_w4, qst1_w4_files = data_handler.load_images(folder = "../data/Test/qst1_w4_denoised/", extension = ".jpg", desc = "Loading qst1_w4 Data...")

# Load text:
db_text = data_handler.load_text(folder = "../data/BBDD/", extension = ".txt", desc = "Loading BBDD Text data...")

Initialized DataHandler with 2 processes


Loading BBDD Data...: 100%|██████████| 287/287 [00:17<00:00, 16.80it/s]


../data/BBDD/ read: 287 images


Loading qst1_w4 Data...: 100%|██████████| 50/50 [00:03<00:00, 15.19it/s]


../data/Test/qst1_w4_denoised/ read: 50 images


Loading BBDD Text data...: 100%|██████████| 287/287 [00:00<00:00, 3869.78it/s]

../data/BBDD/ read: 287 images


In [3]:
from skimage.exposure import adjust_gamma

qst1_w4_filt, qst1_w4_files = data_handler.load_images(folder = "../data/Test/qst1_w4_denoised/", extension = ".jpg", desc = "Loading qst1_w4_denoised Data...")
qst1_w4_filt = [adjust_gamma(img, gamma=2, gain=0.6) for img in qst1_w4_filt]

Loading qst1_w4_denoised Data...: 100%|██████████| 50/50 [00:03<00:00, 14.32it/s]


../data/Test/qst1_w4_denoised/ read: 50 images


In [4]:
from skimage.exposure import adjust_gamma
reload(bg_masks)
res = bg_masks.generate_masks_multiple(
    qst1_w4_filt,
    N_PROCESS=N_PROCESS,
    mssg="Generating masks with up to three components...",
    image_files=qst1_w4_files,
    output_dir='../data/floodFill_masks_qst1_w4/',
    n=3,
)

Generating masks with up to three components...: 100%|██████████| 50/50 [00:48<00:00,  1.04it/s]


In [4]:
# load background masks
masks_gen_floodFill_qst1_w4, masks_gen_floodFill_qst1_w4_files = data_handler.load_images(
    folder=f"../data/floodFill_masks_qst1_w4/",
    extension=".png",
    desc=f"Loading background masks for QST1-W4..."
)

Loading background masks for QST1-W4...: 100%|██████████| 50/50 [00:00<00:00, 58.92it/s]


../data/floodFill_masks_qst1_w4/ read: 50 images


In [5]:
reload(image_utils)
bboxes_qst1_w4, paintings_cropped, mask_text = image_utils.crop_paintings_batch(
    qst1_w4_filt, qst1_w4_files,
    masks_gen_floodFill_qst1_w4,
    output_dir="../results/cropped_paintings_qst1_w4/",
    extract_text_mask=True,
    flag_mask=True,
)
data_handler.save_object_pickle(bboxes_qst1_w4, "./QST1/text_boxes.pkl")

***************
image_id:  00007
Error finding text bounding box: index 0 is out of bounds for axis 0 with size 0
***************
image_id:  00013
Error finding text bounding box: index 0 is out of bounds for axis 0 with size 0
Done cropping paintings.
Object saved at ./QST1/text_boxes.pkl


In [15]:
reload(text_retrieval)
pred_texts_qst1_w4_folder = './results/text/pred_texts_greys_qst1_w4/'
text_retrieval.generate_text_files_multiple(
    "../results/cropped_paintings_qst1_w4/", pred_texts_qst1_w4_folder, greys=True, binary=False)

../results/cropped_paintings_qst1_w4/00000/0.jpg
../results/cropped_paintings_qst1_w4/00001/0.jpg
../results/cropped_paintings_qst1_w4/00001/1.jpg
../results/cropped_paintings_qst1_w4/00002/0.jpg
../results/cropped_paintings_qst1_w4/00003/0.jpg
../results/cropped_paintings_qst1_w4/00004/0.jpg
../results/cropped_paintings_qst1_w4/00005/0.jpg
../results/cropped_paintings_qst1_w4/00005/1.jpg
../results/cropped_paintings_qst1_w4/00005/2.jpg
../results/cropped_paintings_qst1_w4/00006/0.jpg
../results/cropped_paintings_qst1_w4/00007/0.jpg
../results/cropped_paintings_qst1_w4/00007/1.jpg
../results/cropped_paintings_qst1_w4/00008/0.jpg
../results/cropped_paintings_qst1_w4/00009/0.jpg
../results/cropped_paintings_qst1_w4/00010/0.jpg
../results/cropped_paintings_qst1_w4/00011/0.jpg
../results/cropped_paintings_qst1_w4/00012/0.jpg
../results/cropped_paintings_qst1_w4/00013/0.jpg
../results/cropped_paintings_qst1_w4/00014/0.jpg
../results/cropped_paintings_qst1_w4/00015/0.jpg
../results/cropped_p

### AKAZE

In [10]:
reload(image_features)
db_keypoints_akaze, db_features_akaze = image_features.orb_batch(
    images=db[:], mssg="Computing akaze from BBDD...")

































































































































































































































































































Computing akaze from BBDD...: 100%|██████████| 287/287 [03:24<00:00,  1.40it/s]


In [11]:
reload(image_features)
keypoints_multiple_akaze, features_multiple_akaze = image_features.orb_multiple(
    files_dir="../results/cropped_paintings_qst1_w4/", mssg="Computing akaze from QST...")




















































Computing akaze from QST...: 100%|██████████| 50/50 [00:49<00:00,  1.02it/s]


## TODO: try out different similarities...

In [12]:
reload(similarities)
n_matches_qst1_w4_akaze = similarities.compute_matches_multiple(
    features_multiple=features_multiple_akaze,
    db_features=db_features_akaze,
    method="FLANN",
    similarity_measure="Hamming",  # this is ignored if method == "FLANN,
    N_PROCESS=5,
)

In [13]:
len(n_matches_qst1_w4_akaze)

50

In [14]:
reload(evaluate)
top_10_multiple = evaluate.get_top_k_multiple(
    n_matches_qst1_w4_akaze, db_files, k=10, match_threshold=0)

In [16]:
predicted_results_10_qst1_w4_multiple = data_handler.store_outputs_multiple(
    outputs=top_10_multiple,
    save=True,
    name_file="",
    path="./results/qst1_w4_multiple")

Results saved at ./results/qst1_w4_multiple/result.pkl
